In [8]:

import os
import re
import pandas as pd
from tqdm import tqdm



# ============================
# PATH 설정
# ============================
RAW_DIR = "../../data/raw"          # 원본 데이터
OUT_DIR = "../../data/processed"    # 전처리 결과 저장

os.makedirs(OUT_DIR, exist_ok=True)

RAW_FILES = {
    "train": "train.csv",
    "dev": "dev.csv",
    "test": "test.csv",
}

print("RAW_DIR:", RAW_DIR)
print("OUT_DIR:", OUT_DIR)

def normalize_newlines(text: str) -> str:
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = text.replace("\\n", "\n")
    text = re.sub(r"<br\s*/?>", "\n", text, flags=re.I)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text

def normalize_quotes_symbols(text: str) -> str:
    rep = {
        "“": '"', "”": '"',
        "‘": "'", "’": "'",
        "「": '"', "」": '"',
        "『": '"', "』": '"'
    }
    for k, v in rep.items():
        text = text.replace(k, v)
    text = re.sub(r"[『』《》─━│┃╭╮╰╯]", "", text)
    return text

def clean_utterance(text: str) -> str:
    t = text.strip()

    # 1) 감정 표현 제거
    t = re.sub(r"(ㅋㅋ+|ㅎㅎ+|ㅠㅠ+|ㅜㅜ+)", "", t)

    # 2) 물결 제거
    t = re.sub(r"~{2,}", "", t)

    # 3) 말줄임표 → .
    t = re.sub(r"\.{2,}", ".", t)

    # 4) 문장부호 normalize
    t = re.sub(r"\?{2,}", "?", t)
    t = re.sub(r"!{2,}", "!", t)

    # 5) 잘못된 조합 정리
    t = re.sub(r",\?", "?", t)
    t = re.sub(r",\.", ".", t)
    t = re.sub(r"\?\.", "?", t)
    t = re.sub(r"!\.", "!", t)

    # 6) 반복 단어 정리
    t = re.sub(r"\b(\w+)(\s+\1){2,}\b", r"\1", t)

    # ---------------------------------------------------
    # 🔥 7) 문장 경계 분리: "종결형 + 공백 + 일반 단어" → "." 삽입
    # ---------------------------------------------------
    # 예: "있어요 결국" → "있어요. 결국"
    t = re.sub(
        r"(어요|습니다|합니다|해요|예요|에요|였어요|였습니 다|였던|이죠|죠)(\s+)([가-힣A-Za-z0-9])",
        r"\1. \3",
        t
    )

    # 8) 공백 정리
    t = re.sub(r"\s+", " ", t).strip()

    # 9) 문장 끝 문장부호 강제
    if t and t[-1] not in ".?!":
        t = t + "."

    return t




def preprocess_dialogue_v4(text: str) -> str:
    if not isinstance(text, str):
        text = "" if text is None else str(text)

    t = normalize_newlines(text)
    t = normalize_quotes_symbols(t)

    lines = t.split("\n")
    out_lines = []

    for ln in lines:
        stripped = ln.strip()
        if not stripped:
            continue

        # "#PersonN#:" 형태 감지
        m = re.match(r"^(#Person\d+#:)\s*(.*)$", stripped)
        if m:
            tag = m.group(1)                # "#Person1#:"
            utt = m.group(2)                # 발화
            cleaned = clean_utterance(utt)
            out_lines.append(f"{tag} {cleaned}")   # 공백 강제 삽입
        else:
            cleaned = clean_utterance(stripped)
            out_lines.append(cleaned)

    return "\n".join(out_lines)







RAW_DIR: ../../data/raw
OUT_DIR: ../../data/processed


In [9]:
for split, fname in RAW_FILES.items():
    in_path = os.path.join(RAW_DIR, fname)
    out_path = os.path.join(OUT_DIR, f"v4_{split}_preprocessed.csv")

    print(f"[INFO] Preprocessing {split.upper()}  ({in_path} -> {out_path})")
    df = pd.read_csv(in_path)

    df["dialogue"] = [preprocess_dialogue_v4(t) for t in tqdm(df["dialogue"])]

    df.to_csv(out_path, index=False, encoding="utf-8-sig")

print("[INFO] All preprocessing done!")



[INFO] Preprocessing TRAIN  (../../data/raw/train.csv -> ../../data/processed/v4_train_preprocessed.csv)


100%|██████████| 12457/12457 [00:02<00:00, 6133.50it/s]


[INFO] Preprocessing DEV  (../../data/raw/dev.csv -> ../../data/processed/v4_dev_preprocessed.csv)


100%|██████████| 499/499 [00:00<00:00, 6698.82it/s]


[INFO] Preprocessing TEST  (../../data/raw/test.csv -> ../../data/processed/v4_test_preprocessed.csv)


100%|██████████| 499/499 [00:00<00:00, 5900.71it/s]

[INFO] All preprocessing done!


In [10]:
sample_text = """
#Person1#: 안녕하세요!! 피자 주문하실래요??.
#Person2#: 네, #Person1#님. 페퍼로니랑 올리브 추가요!! ㅎㅎㅎ
#Person1#: 대형 피자 할인 있어요~~~ 결국 결국 결국
#Person2#: 오 그런가요,?? 그럼 대형 주세요ㅋㅋㅋㅋㅋㅋㅋ
"""

print("===== BEFORE =====")
print(sample_text)

print("\n===== AFTER (v4 preprocess) =====")
print(preprocess_dialogue_v4(sample_text))


===== BEFORE =====

#Person1#: 안녕하세요!! 피자 주문하실래요??.
#Person2#: 네, #Person1#님. 페퍼로니랑 올리브 추가요!! ㅎㅎㅎ
#Person1#: 대형 피자 할인 있어요~~~ 결국 결국 결국
#Person2#: 오 그런가요,?? 그럼 대형 주세요ㅋㅋㅋㅋㅋㅋㅋ


===== AFTER (v4 preprocess) =====
#Person1#: 안녕하세요! 피자 주문하실래요?
#Person2#: 네, #Person1#님. 페퍼로니랑 올리브 추가요!
#Person1#: 대형 피자 할인 있어요. 결국.
#Person2#: 오 그런가요? 그럼 대형 주세요.
